<a href="https://colab.research.google.com/github/pewikr/hopff/blob/main/Forward_Hopfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def S(x):
    # Define your activation function here
    return 1 / (1 + np.exp(-x))

def dS(x):
    # Define the derivative of your activation function here
    return S(x) * (1 - S(x))

def normalize(input):
    # Normalize the input to the range [0, 1]
    return (input - np.min(input)) / (np.max(input) - np.min(input))

class SimpleLayer:
    def __init__(self, n_neurons, n_inputs, lambda_reg):
        self.weights = np.random.rand(n_neurons, n_inputs)
        self.bias = np.random.rand(n_neurons)
        self.lambda_reg = lambda_reg  # regularization parameter

    def forward(self, input):
        self.input = input
        self.output = S(np.dot(self.weights, input) + self.bias)
        return self.output

    def backward(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights)
        weights_error = np.dot(output_error * dS(self.output), self.input.T) + self.lambda_reg * self.weights
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

    def get_regularization_loss(self):
        # L2 regularization
        return 0.5 * self.lambda_reg * np.sum(np.square(self.weights))

class AssociativeLayer:
    def __init__(self, n_neurons, n_inputs, lambda_reg):
        self.weights = np.random.rand(n_neurons, n_inputs)
        self.weights_ll = np.random.rand(n_neurons, n_neurons)
        self.bias = np.random.rand(n_neurons)
        self.lambda_reg = lambda_reg  # regularization parameter

    def forward(self, input):
        # Compute ACT_temp
        self.input = input
        self.ACT_temp = S(np.dot(self.weights, input))

        # Compute the activation of the layer neurons
        self.output = S(np.dot(self.weights_ll, self.ACT_temp) + np.dot(self.weights, input) + self.bias)

        return self.output

    def backward(self, output_error, learning_rate):
        # Compute the derivative of the output
        d_output = output_error * dS(self.output)

        # Compute the error for weights_ll
        weights_ll_error = np.dot(d_output, self.ACT_temp.T) + self.lambda_reg * self.weights_ll

        # Compute the error for ACT_temp
        ACT_temp_error = np.dot(d_output.T, self.weights_ll)

        # Compute the error for weights
        weights_error = np.dot(ACT_temp_error.T * dS(self.ACT_temp), self.input.T) + np.dot(d_output, self.input.T) + self.lambda_reg * self.weights

        # Update the weights and biases
        self.weights -= learning_rate * weights_error
        self.weights_ll -= learning_rate * weights_ll_error
        self.bias -= learning_rate * np.sum(d_output, axis=1, keepdims=True)

        # Compute the error for the input
        input_error = np.dot(ACT_temp_error.T * dS(self.ACT_temp), self.weights)

        return input_error

    def get_regularization_loss(self):
        # L2 regularization
        return 0.5 * self.lambda_reg * (np.sum(np.square(self.weights)) + np.sum(np.square(self.weights_ll)))

class NeuralNetwork:
    def __init__(self, n_i, n_l, n_o, lambda_reg):
        self.associative_layer = AssociativeLayer(n_l, n_i, lambda_reg)
        self.output_layer = SimpleLayer(n_o, n_l, lambda_reg)

    def forward(self, input):
        # Normalize the input
        input = normalize(input)

        # Compute the activation of the associative layer
        associative_activation = self.associative_layer.forward(input)

        # Compute the output
        output = self.output_layer.forward(associative_activation)

        return output

    def compute_loss(self, output, target):
        # Compute the mean squared error loss
        mse_loss = np.mean((output - target) ** 2)

        # Add the regularization loss
        regularization_loss = self.get_total_regularization_loss()

        # Total loss is the sum of the MSE loss and the regularization loss
        total_loss = mse_loss + regularization_loss

        return total_loss

    def backward(self, output, target, learning_rate):
        # Compute the error
        output_error = 2 * (output - target) / output.size

        # Backpropagate the error
        associative_error = self.output_layer.backward(output_error, learning_rate)
        self.associative_layer.backward(associative_error, learning_rate)

    def get_total_regularization_loss(self):
        # Total regularization loss is the sum of the regularization losses of all layers
        return self.associative_layer.get_regularization_loss() + self.output_layer.get_regularization_loss()
